### Reconstruct all events using four GPUs and the CRS2 spherical optimizer running in batch mode

In [1]:
import pickle
import math
import time
import pkg_resources

from functools import partial
from multiprocessing import Process, Pool

import numpy as np
import pandas as pd

from freedom.llh_service.llh_service import LLHService
from freedom.reco import crs_reco
from freedom.reco import summary_df

In [2]:
with open('/home/atfienberg/freedomDataCopy/public_for_aaron/test_events.pkl', 'rb') as f:
    events = pickle.load(f)

In [3]:
allowed_DOMs = np.load(pkg_resources.resource_filename('freedom', 'resources/allowed_DOMs.npy'))
ndoms = len(allowed_DOMs)

### Specify n live points

In [4]:
n_live_points = 97

### Define allowed search ranges and initial box limits

In [5]:
init_pos_range = 50

time_range = (-1000, 0)

log_energy_range = [0, 1.7]

init_range = np.array( [[-init_pos_range, init_pos_range],
                        [-init_pos_range, init_pos_range],
                        [-2*init_pos_range, 2*init_pos_range],
                        [time_range[0], time_range[1]],
                        [0, 2*math.pi],
                        [0, math.pi],
                        [log_energy_range[0], log_energy_range[1]],
                        [log_energy_range[0], log_energy_range[1]]])

# define limits of search range 
param_search_limits = np.array([
    [-200, 200],
    [-250, 250],
    [-800, -200],
    [8000, 11000],
    [0, 2*math.pi],
    [0, math.pi],
    [5, 400],
    [0.1, 400]
]).T

### llh service configuration:

In [6]:
service_conf = {
        "poll_timeout": 1,
        "flush_period": 1,
        "n_hypo_params": 8,
        "n_hit_features": 9,
        "n_evt_features": 2,
        "batch_size" : {
          "n_hypos": 200,
          "n_observations": 6000, 
        },
        "send_hwm": 10000,
        "recv_hwm": 10000,
#         "hitnet_file": "/home/atfienberg/freedomDataCopy/public_for_aaron/HitNet_ranger_30_Jul_2020-15h49/epoch_32_model.hdf5",
#         "domnet_file": "/home/atfienberg/freedomDataCopy/public_for_aaron/DOMNet_reduced_22_Jul_2020-15h18/epoch_30_model.hdf5",
        "hitnet_file": "/home/atfienberg/freedomDataCopy/public_for_aaron/HitNet_ranger_14_Jul_2020-08h03/epoch_32_model.hdf5",
        "chargenet_file": "ChargeNet_nChannels_22_May_2020-11h05/epoch_400_model.hdf5",
        "ndoms": ndoms,
        "boundary_guard": {
                           "file": "/home/atfienberg/freedomDataCopy/public_for_aaron/boundary_guard_softplus_17_Dec_2020/test_model_softplus_poisson.hdf5",
                           "bg_lim": 1500,
                           "invalid_llh": 1e9,
        }
}

In [7]:
# add hit_data, evt_data keys based on the networks being used
# for now, support domnet and chargenet
for event in events:
    if service_conf['n_hit_features'] == 8:
        event['hit_data'] = event['hits'][:, :8]
    else:
        event['hit_data'] = event['hits']
        
    if 'domnet_file' in service_conf:
        event['evt_data'] = event['doms'][allowed_DOMs]
    else:
        event['evt_data'] = event['total_charge']

### Build four services, one per GPU

In [8]:
n_gpus = 4

In [9]:
base_req = "ipc:///tmp/atfrecotestreq"
base_ctrl = "ipc:///tmp/atfrecotestctrl"

req_addrs = []
ctrl_addrs = []
for i in range(n_gpus):
    req_addrs.append(f'{base_req}{i}')
    ctrl_addrs.append(f'{base_ctrl}{i}')

In [10]:
print(req_addrs)
print(ctrl_addrs)

['ipc:///tmp/atfrecotestreq0', 'ipc:///tmp/atfrecotestreq1', 'ipc:///tmp/atfrecotestreq2', 'ipc:///tmp/atfrecotestreq3']
['ipc:///tmp/atfrecotestctrl0', 'ipc:///tmp/atfrecotestctrl1', 'ipc:///tmp/atfrecotestctrl2', 'ipc:///tmp/atfrecotestctrl3']


In [11]:
procs = []
for i in range(n_gpus):
    proc = Process(target=crs_reco.start_service, args=(service_conf, ctrl_addrs[i], req_addrs[i] ,i))
    proc.start()
    procs.append(proc)

starting service work loop for gpu 0...
starting service work loop for gpu 1...
starting service work loop for gpu 3...
starting service work loop for gpu 2...
Received die command... flushing and exiting
cleaning up
Received die command... flushing and exiting
cleaning up
Received die command... flushing and exiting
cleaning up
Received die command... flushing and exiting
cleaning up


### Fit a single event

In [12]:
%%time
test_out = crs_reco.fit_events(events[:1], 0, ctrl_addrs, init_range, 
                               param_search_limits, n_live_points,
                               batch_size=12, spherical_indices=[[4,5]])

CPU times: user 520 ms, sys: 20 ms, total: 540 ms
Wall time: 9.89 s


In [13]:
print(test_out[0][0]['fun'])
print(test_out[0][1])
print(test_out[0][2])
print('---')
print(test_out[0][0]['n_calls'])
print(test_out[0][0]['nit'])

-26.799736
-20.648993
-19.15583
---
4035
200


In [14]:
# events_to_process = len(events)
events_to_process = 400
pool_size = 200
evts_per_proc = int(math.ceil(events_to_process/pool_size))
evt_splits = [events[i*evts_per_proc:(i+1)*evts_per_proc] for i in range(pool_size)]
print(sum(len(l) for l in evt_splits))

400


In [15]:
gpu_inds = np.arange(pool_size) % n_gpus

In [16]:
fit_events_partial = partial(
        crs_reco.fit_events,
        ctrl_addrs=ctrl_addrs,
        init_range=init_range,
        search_limits=param_search_limits,
        n_live_points=n_live_points,
        spherical_indices=[[4,5]],
        batch_size=12,
)

In [17]:
%%time
start = time.time()
# reconstruct with a worker pool; one LLH client per worker
with Pool(pool_size) as p:
    outs = p.starmap(fit_events_partial, zip(evt_splits, gpu_inds))
delta = time.time() - start

CPU times: user 488 ms, sys: 3.29 s, total: 3.78 s
Wall time: 44.3 s


In [18]:
print(f'measured time: {delta/60:.1f} minutes')

measured time: 0.7 minutes


In [19]:
print(sum(len(out) for out in outs))

400


In [20]:
n_params = len(events[0]['params'])

In [21]:
all_outs = sum((out for out in outs), [])

Note: the following timing is from running on four Titan X GPUs in parallel

In [22]:
total_calls = sum(out[0]['n_calls'] for out in all_outs)
total_iters = sum(out[0]['nit'] for out in all_outs)
print(f'{total_calls} total calls')
time_per_call = delta/total_calls
print(f'{total_iters} total iters')
time_per_iter = delta/total_iters
print(f'{total_calls/len(all_outs):.1f} calls per event')
print(f'{time_per_call*1e6:.2f} us per call')

print(f'{total_iters/len(all_outs):.1f} iters per event')
print(f'{time_per_iter*1e6:.2f} us per iter')

1945719 total calls
96700 total iters
4864.3 calls per event
22.78 us per call
241.8 iters per event
458.38 us per iter


In [23]:
par_names = ['x', 'y', 'z', 'time',
             'azimuth', 'zenith', 
             'cascade energy', 'track energy']
df = summary_df.build_summary_df(all_outs, par_names)

In [24]:
free_f_better = df[df.free_fit_llh < df.retro_p_llh + 10]
frac = len(free_f_better)/len(df)
print(f'free fit better frac: {frac:.2f}')

free fit better frac: 0.99


In [25]:
bad = df[df.free_fit_llh >= df.retro_p_llh + 5]

In [26]:
bad.head()

,evt_idx,free_fit_llh,true_p_llh,retro_p_llh,n_calls,n_iters,x,y,z,time,azimuth,zenith,cascade energy,track energy
54,54,-2.435423e+01,-2.981248e+01,-33.349403,3860,196,81.918444,-168.011801,-373.009579,9403.311885,4.630806,2.337713,26.858976,0.718241
260,260,-1.400080e+02,-2.717598e+02,-288.033752,5356,282,-106.846230,-175.500782,-369.971921,9286.698561,3.760740,1.136570,140.987643,49.344229
296,296,1.000000e+10,-3.776003e+01,-41.822254,97,0,89.847350,-18.710206,-238.421001,12496.720319,4.697346,0.249999,23.159452,6.859718
305,305,-2.827037e+02,-2.698645e+02,-288.172119,5137,265,32.021592,-29.743713,-396.006449,9710.709720,1.596842,2.245904,5.056162,18.650123
318,318,-8.392945e+01,1.000000e+09,-110.495117,3019,160,120.232529,-134.267749,-205.578426,9280.817559,4.087020,0.793031,31.100011,47.157038


In [27]:
df.to_pickle('./test_out.pkl')

In [28]:
# kill all the services
import zmq
for proc, ctrl_addr in zip(procs, ctrl_addrs): 
    with zmq.Context.instance().socket(zmq.REQ) as ctrl_sock:
        ctrl_sock.connect(ctrl_addr)
        ctrl_sock.send_string("die")
        proc.join()